In [ ]:
import pandas as pd
import obspy
import matplotlib.pyplot as plt
from datetime import timezone

import numpy as np
from PIL import Image

In [ ]:
fin_dq = pd.read_csv(
    '/network/projects/aia/whale_call/DATA_QUALITY/fin/fin_quality.csv',
    header=0
    )

In [ ]:
ds = pd.read_csv('/network/projects/aia/whale_call/LABELS/FW/fw_raw.csv')

# Plot matching WAV and Spectrograms

In [ ]:
for index, row in fin_dq.iterrows():

    date = row['date']
    station = row['station']
    component = row['component']

    datetime_start = date + " " + row['t_start']
    datetime_end = date + " " + row['t_end']
    print("=========================== Start : {} | End : {} ===========================".format(datetime_start,datetime_end))
    print("Index:",index)
    print("Station:",station)
    print("date:",date)
    print("component:",component)


    folder = date[:4]+date[5:7]+date[8:10]
    file = date[:4]+"."+date[5:7]+"."+date[8:10]+".CN."+station+".."+component+".SAC"
    
    st = obspy.read(
        '/network/projects/aia/whale_call/FILT_10_32/'+folder+"/"+file,
    )
    start_time=obspy.UTCDateTime(datetime_start)
    end_time=obspy.UTCDateTime(datetime_end)

    st_1 = st.slice(
        start_time, 
        end_time
        )

    tr = st_1[0]

    img = "/network/projects/aia/whale_call/DATA_QUALITY/"+row['whale_type']+"/solo/"+str(index+1)+".png"
    
    """
    dss = ds[(ds.time_call_start > datetime_start) \
        & (ds.time_call_start < datetime_end) \
            & (ds.component == component)\
                & (ds.station_code == station)].copy()
    """

    dss =  ds[(pd.to_datetime(ds.time_call_start) > pd.to_datetime(datetime_start)) & (pd.to_datetime(ds.time_call_start) < pd.to_datetime(datetime_end)) & (ds.component == component) & (ds.station_code == station)].copy()


    dss['time_call_start_dt'] = pd.to_datetime(dss.time_call_start)
    dss['time_call_end_dt'] = pd.to_datetime(dss.time_call_end)

    dss['datetime_start_tz'] = dss.time_call_start_dt.apply(lambda x : x.replace(tzinfo=timezone.utc))
    dss['datetime_end_tz'] = dss.time_call_end_dt.apply(lambda x : x.replace(tzinfo=timezone.utc))

    # Create a figure and axes object
    fig, ax = plt.subplots(nrows=2, figsize=(8, 8))
    ax[0].autoscale(tight=True)

    # Plot the cosine function
    ax[0].plot(tr.times("matplotlib"), tr.data, "b-")

    if dss.shape[0] > 0:
    # Plot whales detections on timeline
        for index,row in dss.iterrows():
            ax[0].axvline(
                x=row['datetime_start_tz'],
                color='g',
                label="whale Call")
            
            ax[0].axvline(
                x=row['datetime_end_tz'],
                color='r',
                label="whale Call")

    ax[0].set_title('Filtered SAC')

    # Load and show the image
    image = Image.open(img)
    ax[1].imshow(image)
    ax[1].set_title('Spectrogram')

    # Adjust the layout
    plt.tight_layout()

    # Display the plot
    plt.show()     

# Compare individual calls

In [ ]:
for index, row in fin_dq.loc[[15]].iterrows():

    date = row['date']
    station = row['station']
    component = row['component']

    datetime_start = date + " " + row['t_start']
    datetime_end = date + " " + row['t_end']
    print("Start : {} | End : {}".format(datetime_start,datetime_end))


    folder = date[:4]+date[5:7]+date[8:10]
    file = date[:4]+"."+date[5:7]+"."+date[8:10]+".CN."+station+".."+component+".SAC"
    
    st = obspy.read(
        '/network/projects/aia/whale_call/FILT_10_32/'+folder+"/"+file,
    )
    start_time=obspy.UTCDateTime(datetime_start)
    end_time=obspy.UTCDateTime(datetime_end)

    st_1 = st.slice(
        start_time, 
        end_time
        )

    tr = st_1[0]
    total_npts = tr.stats.npts

    img = "/network/projects/aia/whale_call/DATA_QUALITY/"+row['whale_type']+"/solo/"+str(index+1)+".png"
    dss =  ds[(pd.to_datetime(ds.time_call_start) > pd.to_datetime(datetime_start)) & (pd.to_datetime(ds.time_call_start) < pd.to_datetime(datetime_end)) & (ds.component == component) & (ds.station_code == station)].copy()


    dss['time_call_start_dt'] = pd.to_datetime(dss.time_call_start)
    dss['time_call_end_dt'] = pd.to_datetime(dss.time_call_end)

    dss['datetime_start_tz'] = dss.time_call_start_dt.apply(lambda x : x.replace(tzinfo=timezone.utc))
    dss['datetime_end_tz'] = dss.time_call_end_dt.apply(lambda x : x.replace(tzinfo=timezone.utc))

    # Create a figure and axes object
    fig, ax = plt.subplots(nrows=2, figsize=(8, 8))
    ax[0].autoscale(tight=True)

    # Plot the cosine function
    ax[0].plot(tr.times("matplotlib"), tr.data, "b-")

    if dss.shape[0] > 0:
    # Plot whales detections on timeline
        for index,row in dss.iterrows():
            ax[0].axvline(
                x=row['datetime_start_tz'],
                color='g',
                label="whale Call")
            
            ax[0].axvline(
                x=row['datetime_end_tz'],
                color='r',
                label="whale Call")

    ax[0].set_title('Filtered SAC')

    # Load and show the image
    image = Image.open(img)
    ax[1].imshow(image)
    ax[1].set_title('Spectrogram')

    # Adjust the layout
    plt.tight_layout()

    # Display the plot
    plt.show()

    for index,row in dss.iloc[2:,:].iterrows():

        st_2 = st.slice(
            obspy.UTCDateTime(row['time_window_start']),
            obspy.UTCDateTime(row['time_window_end'])
        )
        tr = st_2[0]

        # Create a figure and axes object
        fig, ax = plt.subplots(nrows=2, figsize=(8, 8))
        ax[0].autoscale(tight=True)

        # Plot the cosine function
        ax[0].plot(tr.times("matplotlib"), tr.data, "b-")

        ax[0].axvline(
            x=row['datetime_start_tz'],
            color='g',
            label="whale Call")
        
        ax[0].axvline(
            x=row['datetime_end_tz'],
            color='r',
            label="whale Call")

        from PIL import Image
        import numpy as np
        
        # Load image
        image = Image.open(img)
        # Convert to np array
        image_arr = np.array(image)

        # dimensions
        hauteur = image_arr.shape[0]
        longueur = image_arr.shape[1]

        total_wav = len(tr.data)

        from datetime import datetime
        d1 = datetime.strptime(datetime_start, "%Y-%m-%d %H:%M:%S")
        d2 = datetime.strptime(datetime_end, "%Y-%m-%d %H:%M:%S")
        d3 = datetime.strptime(row['time_window_start'], "%Y-%m-%d %H:%M:%S.%f")
        d4 = datetime.strptime(row['time_window_end'], "%Y-%m-%d %H:%M:%S.%f")

        # difference between dates in timedelta
        diff_start = ((d3-d1).total_seconds())
        diff_end = ((d4-d1).total_seconds())
        second_block = abs(longueur / (d1 - d2).total_seconds())

        # Crop image
        image_arr = image_arr[
            :, # Hauteur
            round(diff_start*second_block):round(diff_end*second_block) # Longueur
        ]

        # Convert array to image
        image = Image.fromarray(image_arr)

        new_image = image.resize((1500, 400))
        ax[1].imshow(new_image) 
        plt.show()
